#### NX-414: Brain-like computation and intelligence
##### TA: Alessandro Marin Vargas

# Challenge part 2 - VOneCornNEt

## It was worth a trial - but we had not time to conclude

In [3]:
import sys
#!{sys.executable} -m pip install gdown h5py

In [5]:
from utils import load_it_data, visualize_img
import matplotlib.pyplot as plt
import numpy as np
import gdown
url = "https://drive.google.com/file/d/1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS/view?usp=share_link"
output = "IT_data.h5"
gdown.download(url, output, quiet=False, fuzzy=True)


Downloading...
From: https://drive.google.com/uc?id=1s6caFNRpyR9m7ZM6XEv_e8mcXT3_PnHS
To: /home/jupyter/week 5/IT_data.h5
100%|██████████| 384M/384M [00:01<00:00, 214MB/s]  


'IT_data.h5'

### Load the data

In [ ]:
path_to_data = '' ## Insert the folder where the data is, if you download in the same folder as this notebook then leave it blank

stimulus_train, stimulus_val, stimulus_test, objects_train, objects_val, objects_test, spikes_train, spikes_val = load_it_data(path_to_data)


In [ ]:
n_stimulus, n_channels, img_size, _ = stimulus_train.shape
_, n_neurons = spikes_train.shape
print('The train dataset contains {} stimuli and {} IT neurons'.format(n_stimulus,n_neurons))
print('Each stimulus have {} channgels (RGB)'.format(n_channels))
print('The size of the image is {}x{}'.format(img_size,img_size))

In [ ]:
stim_idx = 1

visualize_img(stimulus_train,objects_train,stim_idx)

In [ ]:
neuron_idx = 1

plt.figure()
plt.title('Average firing rate for neuron {} (70-170 ms)'.format(neuron_idx))
plt.plot(spikes_train[:,neuron_idx])

## Packtage and setup

In [ ]:
#import packtage

import torch
import torchvision
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, Dataset
from math import *
from sklearn import metrics as ms

from sklearn.decomposition import PCA, IncrementalPCA

In [ ]:
import vonenet

In [ ]:
v1_model = vonenet.get_model(model_arch='cornets', pretrained=True).module

print(v1_model)

In [ ]:
print(v1_model)

In [ ]:
import torchvision.transforms as transforms
# The VOneModel was fed with normlalized images as shown below
# Define the mean and std for normalization
mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

#stimulus_train1 = stimulus_train.reshape(2592,64,-1)
#stimulus_val1 = stimulus_val.reshape(288,64,-1)

images_ = torch.from_numpy(stimulus_train)
images2_ = torch.from_numpy(stimulus_val)

# Create a normalization transform
normalize = transforms.Normalize(mean=mean, std=std, inplace=False)

# Apply the normalization transform to the tensor of images
stimulus_train_norm = normalize(images_)

stimulus_val_norm = normalize(images2_)




In [ ]:
#choose the model
#model = torchvision.models.resnet50(weights = ResNet50_Weights.IMAGENET1K_V2)
#model = torchvision.models.resnet50(weights=None)
model = v1_model.model

In [ ]:
#do the batch
dataloader_train = DataLoader(stimulus_train_norm, batch_size = 64,shuffle=True, pin_memory=True)
dataloader_valid = DataLoader(stimulus_val_norm, batch_size = 64,shuffle=True, pin_memory=True)

In [ ]:
#environment for Cornet backend
Name = ['V2','V4','IT']
model_layer = {'V2':model.V2,'V4':model.V4,'IT':model.IT}

correlation = {'V2':[],'V4':[],'IT':[]}
variance = {'V2':[],'V4':[],'IT':[]}


## function to apply the model

In [ ]:
features = []
#function to hook the features
def get_features():
    def hook(model, input, output):
        features.append(output.detach())
    return hook

In [ ]:
#function that get the PCs from a layer (with the training set)
def get_layer(name) :
    hooked =model_layer[name].register_forward_hook(get_features())
    
    for batch in dataloader_train:
        pred_res = v1_model(torch.Tensor(batch))
    
    hooked.remove()
    
    output = torch.concat(features,axis=0).detach().numpy()
    pca.fit(output.reshape(output.shape[0],-1))
    
    return pca.transform(output.reshape(output.shape[0],-1))

In [ ]:
#fonction that get the PCs from a layer (with the validation dataset)
def get_validation(name) :
    hooked =model_layer[name].register_forward_hook(get_features())
    
    for batch in dataloader_valid:
        pred_res = v1_model(torch.Tensor(batch))
    
    hooked.remove()

    output = torch.concat(features,axis=0).detach().numpy()
    
    return pca.transform(output.reshape(output.shape[0],-1))

## function for model evaluation

In [ ]:
#value is the expected neural activity and prediction is the predicted one
#this function plot some neural activities and compute the correlation and explained variance 
#display the mean of correlation and explained variance and return two array

def evaluation(value, prediction, title) :
    
    #plot neural activities
    plt.figure()
    
    N = 5
    fig, axs = plt.subplots(N, 1)

    for n in range(N) : 
        axs[n].plot(value[:,n])
        axs[n].plot(prediction[:,n])

    plt.show()
    plt.savefig('figure/bunch of prediction vs recording neural activity '+ title)

    #compute correlation and explained variance
    correlation = []
    var = []

    for i in range(value.shape[1]) : #each neuron
        correlation.append(pearsonr(value[:,i],prediction[:,i])[0])
        var.append(ms.explained_variance_score(value[:,i],prediction[:,i]))
    
    #plot correlation and explained variance
    plt.figure()
    plt.title('explained variance for each neuron '+ title)
    plt.xlabel('neurons')
    plt.ylabel("explained variance")
    plt.plot(var)
    plt.show()
    plt.savefig('figure/explained variance for each neuron '+ title)
    
    plt.figure()
    plt.title('correlation for each neuron '+ title)
    plt.xlabel('neurons')
    plt.ylabel("correlation")
    plt.plot(correlation)
    plt.show()
    plt.savefig('figure/correlation for each neuron '+ title)
    
    mCorr =np.mean(correlation)
    mVar =np.mean(var)
    #print the means
    print('mean of correlation ', mCorr)
    print('mean of explained variance ',mVar)
    
    #return arrays
    return correlation,var,mCorr,mVar

## model of part 1

In [ ]:
from sklearn import linear_model
from scipy.stats import pearsonr

In [ ]:
LinearModel = linear_model.Ridge()

## Predict neural activity layer by layer

In [ ]:
for layer in Name :
    print("On layer "+layer)
    
    pca = PCA(n_components=1000)
    features = []
    PC_activation = get_layer(layer)
    print("Done 1000 PCs for each layer activation")
    
    features = []
    PC_activation_validation = get_validation(layer)
    print("Done 1000 PCs for prediction")
    
    LinearModel.fit(PC_activation,spikes_train)
    y_pred = LinearModel.predict(PC_activation_validation)
    correlation[layer],variance[layer] = evaluation(spikes_val, y_pred, "VOneNet "+layer)
    

The output does not correspond to the input shape - should be investigated